
# Getting Started with PSBlueSky

An overview of Jeff Hick's cool PSBlueSky module, from installation to posting from the Command Line. (plus evrything is cooler in the CLI ;) )

## How to Install

Can grab it from the PowerShell Gallery via Import-module or Install-PSFModule (and for the PSFRamework.nuget users: Install-PSFModule)

In [ ]:
Install-Module -Name PSBluesky -Scope CurrentUser -Force -AllowClobber

In [ ]:
Install-PSResource -Name PSBluesky -Repository PSGallery -TrustRepository

In [ ]:
Install-PSFModule -Name PSBlueSky

## BlueSky Credentials

Jeff wisely recomends you store your account credentials you need to get from  https://bsky.app/ and store in a vault

• Microsoft.PowerShell.SecretManagement - https://github.com/powershell/secretmanagement
• Microsoft.PowerShell.SecretStore - https://github.com/powershell/secretstore

I have a keepass one setup, can find out more in my blog (advise following Jeff's suggestion)

Powershell Profiles Keepass And Psbluesky | PowerShell Young Team
https://powershellyoungteam.github.io/2024/11/22/PowerShell-Profiles-KeePass-and-PSBlueSky.html

NOTE: Username is case sensitive too!, also don’t be like me and put the @ sign before the username, so like PoshYoungTeam.bsky.social, NOT @PoshYoungTeam.bsky.social

if you use 2FA you will need to create an App password, there is detailed instructions in the help pdf.

In [ ]:
import-module PSBlueSky
Import-Module SecretManagement.KeePass

$BskyCreds = Get-Secret -Vault PowerShell -Name BlueSky


Keepass Master Password
Enter the Keepass Master password for: C:\Users\Wighty\OneDrive\Documents\SecVault\PowerShell.kdbx


## Help!

PSBlueSky has some Gucci built in help! check out these Cmdletts as well as Github help and built in Help (get-Help)

In [1]:
Open-BskyHelp # which will open a PDF version of this document in your default PDF viewer

Get-BskyModuleInfo # which will show you the module commands and has clickable links to online help and module's github page


   Module: ]8;;https://github.com/jdhitsolutions/PSBluesky\PSBlueSky]8;;\ [v2.5.0]

Name                      Alias               Synopsis
----                      -----               --------
]8;;https://github.com/jdhitsolutions/PSBluesky/blob/main/docs/Add-BskyImage.md\Add-BskyImage]8;;\                                 Upload an image to Bluesky.
]8;;https://github.com/jdhitsolutions/PSBluesky/blob/main/docs/Block-BskyUser.md\Block-BskyUser]8;;\                                Block a Bluesky user account.
]8;;https://github.com/jdhitsolutions/PSBluesky/blob/main/docs/Disable-BskyLogging.md\Disable-BskyLogging]8;;\                           Disable BlueSky API logging.
]8;;https://github.com/jdhitsolutions/PSBluesky/blob/main/docs/Enable-BskyLogging.md\Enable-BskyLogging]8;;\                            Enable BlueSky API logging.
]8;;https://github.com/jdhitsolutions/PSBluesky/blob/main/docs/Export-BskyPreference.md\Export-BskyPreference]8;;\                

Error: This command cannot find a matching alias because an alias with the definition 'Unblock-BskyUser' does not exist.
This command cannot find a matching alias because an alias with the definition 'Start-BSkySession' does not exist.
This command cannot find a matching alias because an alias with the definition 'Set-BskyPreference' does not exist.
This command cannot find a matching alias because an alias with the definition 'Set-BskyLogging' does not exist.
This command cannot find a matching alias because an alias with the definition 'Remove-BskyPreferenceFile' does not exist.
This command cannot find a matching alias because an alias with the definition 'Remove-BskyLogging' does not exist.
This command cannot find a matching alias because an alias with the definition 'Get-BskyPreference' does not exist.
This command cannot find a matching alias because an alias with the definition 'Get-BskyModuleInfo' does not exist.
This command cannot find a matching alias because an ali

## Start a session

you need to use the creds in your vault!

you can create a session, and if you get a timeout error, you can refresh the session.


In [ ]:
Start-BskySession -credential $cred

Get-BskySession # confirm that you are connected to the session

Get-BSkySession | Update-BskySession # to update the session



In [ ]:
# Posting! this was my first post..

New-BskyPost -message "It works!! :)" -imagepath C:\users\Wighty\Pictures\BskyPic.PNG -ImageAlt "pic of my first PSBlueSky post" -Verbose

# pass as an object

$param = @{
    Message = "It works!! :)"
    ImagePath = "C:\users\Wighty\Pictures\BskyPic.PNG "
    ImageAlt = "pic of my first PSBlueSky post"
    Verbose = $true
}

New-BskyPost @param

# with a URL, make sure URL is surrounded by Whitespace

New-BskyPost "Cool #PowerShell tip popped up in my terminal today from Daniel Schroeder ( @deadlydog.bsky.social ). In short, watch this Github repo to stay up to date with known PwSh security vulnerabilities and breaking changes:   [PowerShell Annoucements] (https://github.com/PowerShell/Announcements) " -Verbose

# multiline post, save text as string and pass to New-BskyPost

$multiline = @"
I'm really enjoying the #PSBlueSky module. It's a great way to post to Bluesky from #PowerShell. I can't wait to see what other features are added in the future.

#PowerShell
#PSBlueSky
"@

New-BskyPost $multiline -Verbose -WhatIf

In [ ]:
# Feeds and Followers

Get-BskyFeed -Verbose # get your feed (good to filter down by time or limit)

Get-BskyTimeline -Verbose # get your timeline (good to filter down by time or limit)

Get-BskyNotification -Verbose # get your notifications with clickable links to the post

# reposting, need non -default properties URI and CID

Get-BskyTimeline -Limit 1 | Select-Object Author,Date,Text,URI, CID | Publish-BskyPost -Quote "Testing quoting with a PSBluesky command" -whatif

In [ ]:
# User Stuff

Find-BskyUser -UserName "jsnover.com"  # find anyone with PowerShell in their username and Description

Find-BskyUser -UserName "powershell" | Select-Object username # use this to get a list of usernames for demo purposes :)

Get-BskyProfile # Get your profile/bio or pass another users username

Get-BskyFollowers # get your followers

# see how big the Young Team is

$YoungTeamers = Get-BskyFollowers
Write-Output "You have $($YoungTeamers.Count) in the Young Team"

#Let's visit the Young Team

#First get the path to my MSedge.exe
$EdgePath = 'C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe'
$YoungTeamers | ForEach-Object { Start-Process $EdgePath $_.URL }

Get-BskyFollowing # get who you are following

# Following, Unfollowing and blocking badies!

New-BskyFollow "PoshYoungTeam.bsky.social" -WhatIf

#or get pipy with it

Get-BskyProfile "PoshYoungTeam.bsky.social"  | New-BskyFollow -passthru

Remove-BskyFollow "PoshYoungTeam.bsky.social" -WhatIf

#or get pipy with it

Get-BskyProfile "PoshYoungTeam.bsky.social" | Remove-BskyFollow -passthru -WhatIf

#blocking

Get-BskyBlockedUser # get who you have blocked

Block-BskyUser -UserName "baduser" -WhatIf

Unblock-BskyUser -UserName "baduser" -WhatIf


6. Resources

• GitHub - PSBlueSky - https://github.com/jdhitsolutions/PSBluesky
Bluesky API Documentation - [HTTP Reference | Bluesky  ](https://docs.bsky.app/docs/category/http-reference)

https://github.com/jdhitsolutions/PSBluesky/discussions - checking this out for other uses!!!

and see what people say here:



‪Jeff Brown Tech‬ ‪@jeffbrowntech.bsky.social‬

Using it in Azure Functions to call it from a Logic App for posting new blogs and videos to social.

I've shared the ways I'm using the PSBlueSky #PowerShell module in the repo's Discussion section. How are you using
the module? jeffhicks bluesky https://bsky.app/profile/did:plc:ohgsqpfsbocaaxusxqlgfvd7/post/3li3j3yqxbh23